In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class ZFNet(nn.Module):
    def __init__(self, input_channel=3, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channel, 96, 7, stride=2, padding=0)
        self.bn1 = nn.BatchNorm2d(96)
        self.conv2 = nn.Conv2d(96, 256, 3, stride=2, padding=0)
        self.bn2 = nn.BatchNorm2d(256)
        self.conv3 = nn.Conv2d(256, 384, 3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(384)
        self.conv4 = nn.Conv2d(384, 384, 3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(384)
        self.conv5 = nn.Conv2d(384, 256, 3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(256*5*5, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)

    def forward(self, x):
        # N x 3 x 224 x 224
        x = F.max_pool2d(F.relu(self.bn1(self.conv1(x))), 3, stride=2)
        # N x 96 x 54 x 54
        x = F.max_pool2d(F.relu(self.bn2(self.conv2(x))), 3, stride=2)
        # N x 256 x 12 x 12
        x = F.relu(self.bn3(self.conv3(x)))
        # N x 384 x 12 x 12
        x = F.relu(self.bn4(self.conv4(x)))
        # N x 384 x 12 x 12
        x = F.max_pool2d(F.relu(self.bn5(self.conv5(x))), 3, stride=2)
        # N x 256 x 5 x 5
        x = x.view(-1, 256*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
x = torch.randn(1, 3, 224, 224)
model = ZFNet()
model(x)

tensor([[-0.1462, -0.1591, -0.0085,  0.1022,  0.0988, -0.0987,  0.0224, -0.2334,
          0.0781, -0.0382]], grad_fn=<AddmmBackward0>)